##COVID-19 Prediction Model

Derek Cheung & Padrick Beggs

In [1]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


This is a link to the download of the data set on kaggle. The dataset is updated at the end of each day. 

https://www.kaggle.com/sudalairajkumar/novel-corona-virus-2019-dataset#time_series_covid_19_confirmed.csv




In [0]:
#country check, will use number of countries as first layer of neural network
dfConfirmed = pd.read_csv("time_series_covid_19_confirmed.csv")
countries = {}
for country in dfConfirmed['Country/Region']:
    if country in countries:
        countries[country] += 1
    else:
        countries[country] = 1
print(str(len(countries)) + ' countries')

176 countries


In [0]:
#Restructuring dataset

dfConfirmed = dfConfirmed.drop(['Province/State','Lat', 'Long'], axis = 1)
dfConfirmed.loc['Sum Cases'] = dfConfirmed.sum()
dfConfirmed['Country/Region']['Sum Cases'] = 'Global Confirmed Cases'
dfConfirmed = dfConfirmed.fillna(value = 0)
dfConfirmed.tail()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20
245,Canada,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
246,Canada,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,3
247,Kosovo,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,71,86
248,Burma,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8
Sum Cases,Global Confirmed Cases,555,654,941,1434,2118,2927,5578,6166,8234,9927,12038,16787,19881,23892,27635,30794,34391,37120,40150,42762,44802,45221,60368,66885,69030,71224,73258,75136,75639,76197,76819,78572,78958,79561,80406,81388,82746,84112,86011,88369,90306,92840,95120,97886,101801,105847,109821,113590,118620,125875,128352,145205,156101,167454,181574,197102,214821,242500,272035,304396,336953,378235,418045,467653,529591,593291


In [0]:
#activation function will be either 'relu' or 'selu'
#loss function will be 'mean squared error'

In [0]:
dfConfirmed = dfConfirmed.transpose().drop(['Country/Region'])
dfConfirmed = dfConfirmed.astype(int)
dfConfirmed.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,Sum Cases
1/22/20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,555
1/23/20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,654
1/24/20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,941
1/25/20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1434
1/26/20,0,0,0,0,0,0,0,0,0,3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2118


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
dataArray = dfConfirmed.to_numpy()
print(dataArray)

[[     0      0      0 ...      0      0    555]
 [     0      0      0 ...      0      0    654]
 [     0      0      0 ...      0      0    941]
 ...
 [    84    146    302 ...      0      0 467653]
 [    94    174    367 ...     71      0 529591]
 [   110    186    409 ...     86      8 593291]]


In [0]:
#split a multivariate sequence into samples
from numpy import array
def split_sequences(sequences, n_steps):
  X, y = list(), list()
  for i in range(len(sequences)):
    end_ix = i + n_steps
    if end_ix > len(sequences) - 1:
      break
    seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
    X.append(seq_x)
    y.append(seq_y)
  return(array(X), array(y))

In [0]:
# choose a number of time steps
n_steps = 3
# convert into input/output
X, y = split_sequences(dataArray, n_steps)
n_features = X.shape[2]

In [0]:
print(len(X))
print(len(y))

63
63


In [0]:
# define model
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

In [0]:
X_train, X_test = X[:len(dfConfirmed)-11], X[len(dfConfirmed)-11:]
y_train, y_test = y[:len(dfConfirmed)-11], y[len(dfConfirmed)-11:]

In [0]:
hu1 = 176
hu2 = 150
hu3 = 125
hu4 = 100
hu5 = 75
hu6 = 50
hu7 = 25
afunc = tf.nn.relu

In [0]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv1D(filters = 64, kernel_size = 2, activation = afunc, input_shape=(n_steps, n_features)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(hu1, activation=afunc))
model.add(tf.keras.layers.Dense(hu2, activation=afunc))
model.add(tf.keras.layers.Dense(hu3, activation=afunc))
model.add(tf.keras.layers.Dense(hu4, activation=afunc))
model.add(tf.keras.layers.Dense(hu5, activation=afunc))
model.add(tf.keras.layers.Dense(hu6, activation=afunc))
model.add(tf.keras.layers.Dense(hu7, activation=afunc))
#model.add(tf.keras.layers.Dropout(rate = 0.5))
model.add(tf.keras.layers.Dense(n_features, activation=tf.nn.relu))

In [0]:
optim = 'adam'
model.compile(optimizer=optim,loss='mean_squared_error',metrics=['MeanSquaredError'])

In [0]:
model.fit(X_train, y_train, epochs=3000)

Epoch 1/3000
2/2 [==============================] - 0s 5ms/step - loss: 47735896.0000 - mean_squared_error: 48229116.0000
Epoch 2/3000
2/2 [==============================] - 0s 4ms/step - loss: 46296272.0000 - mean_squared_error: 47198868.0000
Epoch 3/3000
2/2 [==============================] - 0s 5ms/step - loss: 46121028.0000 - mean_squared_error: 45951292.0000
Epoch 4/3000
2/2 [==============================] - 0s 5ms/step - loss: 44141120.0000 - mean_squared_error: 44913528.0000
Epoch 5/3000
2/2 [==============================] - 0s 4ms/step - loss: 42799108.0000 - mean_squared_error: 43935164.0000
Epoch 6/3000
2/2 [==============================] - 0s 4ms/step - loss: 43295136.0000 - mean_squared_error: 42903448.0000
Epoch 7/3000
2/2 [==============================] - 0s 4ms/step - loss: 41295180.0000 - mean_squared_error: 41937552.0000
Epoch 8/3000
2/2 [==============================] - 0s 7ms/step - loss: 39908416.0000 - mean_squared_error: 40674384.0000
Epoch 9/3000
2/2 [======

KeyboardInterrupt: ignored

In [0]:
val_loss, val_acc = model.evaluate(X_test, y_test)
print(val_loss)
print(val_acc)

In [0]:
#3/28/20 prediction
x_input = dfConfirmed.loc[['3/25/20','3/26/20','3/27/20']].to_numpy()
x_input = x_input.reshape((1, n_steps, n_features))
march28 = model.predict(x_input, verbose=0)
print(march28)